In [1]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import sqlite3

In [5]:
data = pd.read_csv('kidney_disease (1).csv')
data.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [6]:
connection = sqlite3.connect('demo.db')

In [8]:
data.to_sql('disease', connection, if_exists = 'replace')

400

In [3]:
data.classification.unique()

array(['ckd', 'ckd\t', 'notckd'], dtype=object)

In [4]:
data.classification = data.classification.replace("ckd\t", "ckd")
data.classification.unique()

array(['ckd', 'notckd'], dtype=object)

In [5]:
data['classification'] = data['classification'].replace(['ckd','notckd'], [1,0])

In [6]:
data.drop('id', axis=1, inplace=True)
data.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44,7800,5.2,yes,yes,no,good,no,no,1
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38,6000,NaN,no,no,no,good,no,no,1
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31,7500,NaN,no,yes,no,poor,no,yes,1
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32,6700,3.9,yes,no,no,poor,yes,yes,1
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35,7300,4.6,no,no,no,good,no,no,1


In [7]:
data.isnull().sum()

age                 9
bp                 12
sg                 47
al                 46
su                 49
rbc               152
pc                 65
pcc                 4
ba                  4
bgr                44
bu                 19
sc                 17
sod                87
pot                88
hemo               52
pcv                70
wc                105
rc                130
htn                 2
dm                  2
cad                 2
appet               1
pe                  1
ane                 1
classification      0
dtype: int64

In [8]:
numerical_columns = ['age', "bp","sg","al","su","bgr",'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc']

data[numerical_columns].isnull().sum()

age       9
bp       12
sg       47
al       46
su       49
bgr      44
bu       19
sc       17
sod      87
pot      88
hemo     52
pcv      70
wc      105
rc      130
dtype: int64

In [9]:
data['pcv'] = data['pcv'].replace('', '0')
data['rc'] = data['rc'].replace('', '0')
data['wc'] = data['wc'].replace('', '0')
data['pcv'] = data['pcv'].replace('\t?', '0')
data['rc'] = data['rc'].replace('\t?', '0')
data['wc'] = data['wc'].replace('\t?', '0')
data['pcv'] = data['pcv'].replace('\t43', '43')
data['wc'] = data['rc'].replace('\t6200', '6200')
data['wc'] = data['wc'].replace('\t8400', '8400')

In [10]:
import numpy as np
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='median')
data[numerical_columns] = imputer.fit_transform(data[numerical_columns])

In [11]:
data.isnull().sum()

age                 0
bp                  0
sg                  0
al                  0
su                  0
rbc               152
pc                 65
pcc                 4
ba                  4
bgr                 0
bu                  0
sc                  0
sod                 0
pot                 0
hemo                0
pcv                 0
wc                  0
rc                  0
htn                 2
dm                  2
cad                 2
appet               1
pe                  1
ane                 1
classification      0
dtype: int64

In [12]:
categorical_columns = ['rbc', 'pc','pcc','ba', 'htn','dm','cad','appet','pe','ane']
data[categorical_columns].isnull().mean()*100

rbc      38.00
pc       16.25
pcc       1.00
ba        1.00
htn       0.50
dm        0.50
cad       0.50
appet     0.25
pe        0.25
ane       0.25
dtype: float64

In [13]:
for i in categorical_columns:
    print(i, ": ", data[i].mode())

rbc :  0    normal
Name: rbc, dtype: object
pc :  0    normal
Name: pc, dtype: object
pcc :  0    notpresent
Name: pcc, dtype: object
ba :  0    notpresent
Name: ba, dtype: object
htn :  0    no
Name: htn, dtype: object
dm :  0    no
Name: dm, dtype: object
cad :  0    no
Name: cad, dtype: object
appet :  0    good
Name: appet, dtype: object
pe :  0    no
Name: pe, dtype: object
ane :  0    no
Name: ane, dtype: object


In [14]:
data['rbc'].fillna('normal', inplace=True)
data['pc'].fillna('normal', inplace=True)
data['pcc'].fillna('notpresent', inplace=True)
data['ba'].fillna('notpresent', inplace=True)
data['htn'].fillna('no', inplace=True)
data['dm'].fillna('no', inplace=True)
data['cad'].fillna('no', inplace=True)
data['appet'].fillna('good', inplace=True)
data['pe'].fillna('no', inplace=True)
data['ane'].fillna('no', inplace=True)

In [15]:
data.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.0,1.020,1.0,0.0,normal,normal,notpresent,notpresent,121.0,...,44.0,5.2,5.2,yes,yes,no,good,no,no,1
1,7.0,50.0,1.020,4.0,0.0,normal,normal,notpresent,notpresent,121.0,...,38.0,4.8,4.8,no,no,no,good,no,no,1
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,4.8,4.8,no,yes,no,poor,no,yes,1
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,3.9,3.9,yes,no,no,poor,yes,yes,1
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,4.6,4.6,no,no,no,good,no,no,1


In [16]:
data['pcv']=data['pcv'].astype(int)
data['wc']=data['wc'].astype(int)
data['rc']=data['rc'].astype(float)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             400 non-null    float64
 1   bp              400 non-null    float64
 2   sg              400 non-null    float64
 3   al              400 non-null    float64
 4   su              400 non-null    float64
 5   rbc             400 non-null    object 
 6   pc              400 non-null    object 
 7   pcc             400 non-null    object 
 8   ba              400 non-null    object 
 9   bgr             400 non-null    float64
 10  bu              400 non-null    float64
 11  sc              400 non-null    float64
 12  sod             400 non-null    float64
 13  pot             400 non-null    float64
 14  hemo            400 non-null    float64
 15  pcv             400 non-null    int64  
 16  wc              400 non-null    int64  
 17  rc              400 non-null    flo

In [17]:
cat_mapping = {
        "rbc": {
        "abnormal":1,
        "normal": 0,
    },
        "pc":{
        "abnormal":1,
        "normal": 0,
    },
        "pcc":{
        "present":1,
        "notpresent":0,
    },
        "ba":{
        "notpresent":0,
        "present": 1,
    },
        "htn":{
        "yes":1,
        "no": 0,
    },
        "dm":{
        "yes":1,
        "no":0,
    },
        "cad":{
        "yes":1,
        "no": 0,
    },
        "appet":{
        "good":1,
        "poor": 0,
    },
        "pe":{
        "yes":1,
        "no":0,
    },
        "ane":{
        "yes":1,
        "no":0,
    }
}

In [18]:
data[categorical_columns] = data[categorical_columns].replace(cat_mapping)

In [19]:
data.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.0,1.020,1.0,0.0,0,0,0,0,121.0,...,44,5,5.2,1,1,0,1,0,0,1
1,7.0,50.0,1.020,4.0,0.0,0,0,0,0,121.0,...,38,4,4.8,0,0,0,1,0,0,1
2,62.0,80.0,1.010,2.0,3.0,0,0,0,0,423.0,...,31,4,4.8,0,1,0,0,0,1,1
3,48.0,70.0,1.005,4.0,0.0,0,1,1,0,117.0,...,32,3,3.9,1,0,0,0,1,1,1
4,51.0,80.0,1.010,2.0,0.0,0,0,0,0,106.0,...,35,4,4.6,0,0,0,1,0,0,1


In [21]:
x = data.drop(['classification', 'sg', 'sod', 'hemo', 'pcv', 'rc', 'appet'], axis=1)
y = data['classification']

In [22]:
x.columns

Index(['age', 'bp', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu', 'sc',
       'pot', 'wc', 'htn', 'dm', 'cad', 'pe', 'ane'],
      dtype='object')

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [24]:
from sklearn.ensemble import RandomForestClassifier

In [26]:
X_test['dm'].unique()
X_train['dm'] = X_train['dm'].replace('\tyes', 1)
X_train['dm'] = X_train['dm'].replace('\tno', 0)
X_train['cad'] = X_train['cad'].replace('\tno', 0)
X_test['dm'] = X_test['dm'].replace(' yes', 1)

In [27]:
model = RandomForestClassifier(n_estimators = 20)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=20)

In [28]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [29]:
confusion_matrix(y_test, model.predict(X_test))

array([[28,  0],
       [ 2, 50]])

In [32]:
print(f"Accuracy is {round(accuracy_score(y_test, model.predict(X_test))*100, 2)}%")

Accuracy is 97.5%


In [35]:
import pickle
pickle.dump(model, open('../kidney.pkl', 'wb'))